In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr

from glob import glob
import seaborn as sns

import numpy as np
import pandas as pd

import warnings

from xskillscore import crps_gaussian, rmse

In [2]:
SECONDS_IN_YEAR = 60*60*24*365 #s
convert = lambda x: x * SECONDS_IN_YEAR * 1e-12 # kg -> Gt

data_path = "../"


# Only future scenarios for now
inputs = glob(data_path + "inputs_s*.nc")

def get_rmse(truth, pred):
    return np.sqrt(((truth-pred)**2))

def global_mean(ds):
    weights = np.cos(np.deg2rad(ds.latitude))
    return ds.weighted(weights).mean(['latitude', 'longitude'])


In [3]:
# variables = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']

# Y = xr.open_dataset(data_path + 'outputs_ssp245.nc').sel(time=slice(2050, 2100))

# # Convert the precip values to mm/day
# Y["pr"] *= 86400
# Y["pr90"] *= 86400

# gp_predictions = xr.merge([{v: xr.open_dataarray(data_path + "outputs_ssp245_predict_gp_{}.nc".format(v))} for v in variables]).sel(time=slice(2050, 2100))
# gp_predictions_std = xr.merge([{v: xr.open_dataarray(data_path + "outputs_ssp245_predict_gp_{}_std.nc".format(v))} for v in variables]).sel(time=slice(2050, 2100))


#-----Original code has discripancies with the actual netcdf file names
variables = ['tas', 'dtr', 'pr', 'pr90']

Y = xr.open_dataset(data_path + 'outputs_ssp245.nc').sel(time=slice(2050, 2100))

# Convert the precip values to mm/day
Y["pr"] *= 86400
Y["pr90"] *= 86400

# Adjusting the filenames to match the available files in the directory
gp_predictions = xr.merge([{v: xr.open_dataarray(data_path + "outputs_ssp245_predict_{}.nc".format(v))} for v in variables]).sel(time=slice(2050, 2100))

# Assuming the standard deviation files follow the same naming pattern as the prediction files
# If this is not the case, you will need to adjust the filenames accordingly
gp_predictions_std = xr.merge([{v: xr.open_dataarray(data_path + "outputs_ssp245_predict_{}_std.nc".format(v))} for v in variables]).sel(time=slice(2050, 2100))


FileNotFoundError: [Errno 2] No such file or directory: '/home/cauliflower/ClimateBench/outputs_ssp245_predict_tas_std.nc'

In [ ]:
for v in variables:
    print(crps_gaussian(Y[v], gp_predictions[v], gp_predictions_std[v], weights=np.cos(np.deg2rad(Y.lat))))